In [4]:
import psycopg2
import sys
from sqlalchemy import create_engine
import geopandas as gpd
import os

In [5]:
DATABASE_NAME = "lab_postgis"
PASSWROD = os.environ.get("POSTGRES_PASS")

In [6]:
def getEngineConnectionString(user='postgres', password=PASSWROD, dbname='postgres'):
    return "postgresql://" + user + ":" + password + "@localhost/" + dbname

In [7]:
def getOpenConnection(user='postgres', password=PASSWROD, dbname='postgres'):
    return psycopg2.connect(database = dbname, user = user, host='localhost', password= password)

In [8]:
def createDB(dbname='postgres'):
    """
    We create a DB by connecting to the default user and database of Postgres
    The function first checks if an existing database exists for a given name, else creates it.
    :return:None
    """
    # Connect to the default database
    con = getOpenConnection(dbname='postgres')
    con.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    cur = con.cursor()

    # Check if an existing database with the same name exists
    cur.execute('SELECT COUNT(*) FROM pg_catalog.pg_database WHERE datname=\'%s\'' % (dbname,))
    count = cur.fetchone()[0]
    if count == 0:
        cur.execute('CREATE DATABASE %s' % (dbname,))  # Create the database
        print("DATABASE " + dbname + " created")
    else:
        print("Database with the given name already exists")

    # Clean up
    cur.close()
    con.commit()
    con.close()

In [9]:
createDB(DATABASE_NAME)

Database with the given name already exists


In [10]:
engine = create_engine(getEngineConnectionString(dbname = DATABASE_NAME))
con = engine.connect()
print("sqlalchemy engine created")

sqlalchemy engine created


In [11]:
try:
    con.execute("CREATE EXTENSION postgis;") # Add PostGIS extension
    print("postgis extension created")
except:
    print("Unable to create postgis extension")

Unable to create postgis extension


In [12]:
dfAirbnb = gpd.read_file("inputs/airbnb")

In [13]:
dfAirbnb.head()

,community,shape_area,shape_len,AREAID,response_r,accept_r,rev_rating,price_pp,room_type,num_spots,...,crowded,dependency,without_hs,unemployed,income_pc,harship_in,num_crimes,num_theft,population,geometry
0,DOUGLAS,46004621.1581,31027.0545098,35,98.771429,94.514286,87.777778,78.157895,1.789474,38,...,1.8,30.7,14.3,18.2,23791,47,5013,1241,18238,"POLYGON ((-87.60914 41.84469, -87.60915 41.844..."
1,OAKLAND,16913961.0408,19565.5061533,36,99.200000,90.105263,88.812500,53.775000,1.850000,20,...,1.3,40.4,18.4,28.7,19252,78,1306,311,5918,"POLYGON ((-87.59215 41.81693, -87.59231 41.816..."
2,FULLER PARK,19916704.8692,None,37,68.000000,NaN,91.750000,84.000000,1.833333,6,...,3.2,44.9,26.6,33.9,10432,97,1764,383,2876,"POLYGON ((-87.62880 41.80189, -87.62879 41.801..."
3,GRAND BOULEVARD,48492503.1554,28196.8371573,38,94.037037,83.615385,92.750000,119.533333,1.533333,30,...,3.3,39.5,15.9,24.3,23472,57,6416,1428,21929,"POLYGON ((-87.60671 41.81681, -87.60670 41.816..."
4,KENWOOD,29071741.9283,23325.1679062,39,92.542857,88.142857,90.656250,77.991453,1.615385,39,...,2.4,35.4,11.3,15.7,35911,26,2713,654,17841,"POLYGON ((-87.59215 41.81693, -87.59215 41.816..."


In [14]:
dfAirbnb.to_postgis("airbnb", engine)#(tablename, connection_object)

ValueError: Table 'airbnb' already exists.

In [15]:
#-------------------------------------------------------------------------------------------------------------------------#

In [16]:
dfGrocery = gpd.read_file("inputs/grocery/Chicago_sup.shp")

In [17]:
dfGrocery.head()

,OBJECTID,Ycoord,Xcoord,Status,Address,Chain,Category,geometry
0,16,41.973266,-87.657073,OPEN,"1051 W ARGYLE ST, CHICAGO, IL. 60640",VIET HOA PLAZA,None,MULTIPOINT (1168268.672 1933554.350)
1,18,41.696367,-87.681315,OPEN,"10800 S WESTERN AVE, CHICAGO, IL. 60643-3226",COUNTY FAIR FOODS,None,MULTIPOINT (1162302.618 1832900.224)
2,22,41.868634,-87.638638,OPEN,"1101 S CANAL ST, CHICAGO, IL. 60607-4932",WHOLE FOODS MARKET,None,MULTIPOINT (1173317.042 1895425.426)
3,23,41.877590,-87.654953,OPEN,"1101 W JACKSON BLVD, CHICAGO, IL. 60607-2905",TARGET/SUPER,new,MULTIPOINT (1168996.475 1898801.406)
4,27,41.737696,-87.625795,OPEN,"112 W 87TH ST, CHICAGO, IL. 60620-1318",FOOD 4 LESS,None,MULTIPOINT (1176991.989 1847262.423)


In [18]:
dfGrocery.to_postgis("grocery", engine)

ValueError: Table 'grocery' already exists.

In [27]:
#close the previous connection and open a new connection where we specify the name of our database
#so this database will be used for further execution
if con:
    con.close()

con = getOpenConnection(dbname = DATABASE_NAME)
cur = con.cursor()

In [20]:
#We need to check if SRIDs of geometry column of both tables are same
#get the SRID for Airbnb table
cur.execute("Select ST_SRID(geometry) from airbnb limit 1")
results = cur.fetchall()
print(results)

[(4326,)]


In [21]:
#get the SRID of grocery table
cur.execute("Select ST_SRID(geometry) from grocery limit 1")
results = cur.fetchall()
print(results)

[(6312,)]


In [22]:
#change SRID for grocery so that both are compatible for further operations
cur.execute("create table grocery3 as select ST_Transform(geometry, 4326) as geometry from grocery")

In [23]:
cur.execute("select ST_AsText(geometry) from grocery3 limit1")
results = cur.fetchall()
print(results)

[('MULTIPOINT(46.06287193273509 48.290595542702505)',), ('MULTIPOINT(45.76229352543728 47.41709723022278)',), ('MULTIPOINT(46.04280440417514 47.94890669715197)',), ('MULTIPOINT(45.993995939785385 47.984991177072665)',), ('MULTIPOINT(45.98353057709055 47.52133875560612)',), ('MULTIPOINT(46.074058080900045 47.94810188985351)',), ('MULTIPOINT(45.81644501604855 47.35664274536489)',), ('MULTIPOINT(45.81513722048567 47.35065201663056)',), ('MULTIPOINT(46.053354109638285 48.06340434233622)',), ('MULTIPOINT(46.09791072194087 48.05995738014233)',), ('MULTIPOINT(45.94406432409869 47.95281675566185)',), ('MULTIPOINT(46.08903073084791 47.93854514727709)',), ('MULTIPOINT(46.142070680629374 47.71140734643572)',), ('MULTIPOINT(46.039908037621515 47.93787916206182)',), ('MULTIPOINT(45.962684089719545 48.08139365869444)',), ('MULTIPOINT(45.87193375596409 47.5321162108365)',), ('MULTIPOINT(45.9191022387973 47.76604098577402)',), ('MULTIPOINT(46.0432586997842 48.37462535698951)',), ('MULTIPOINT(46.172182

In [24]:
cur.execute("Select ST_SRID(geometry) from grocery3 limit 1")
results = cur.fetchall()
print(results)

[(4326,)]


In [25]:
#actual problem with solution
#count how many grocery shops are there in the neighbourhood of an airbnb
cur.execute("CREATE TABLE airbnb_grocery3 AS SELECT airbnb.geometry AS geom1, COUNT(grocery2.geometry) AS grocery_count FROM airbnb, grocery2 WHERE ST_Distance(airbnb.geometry, grocery2.geometry) < 133.9 GROUP BY airbnb.geometry")
#above query has processed the calculation and stored the result in a new table airbnb_grocery

UndefinedTable: relation "grocery2" does not exist
LINE 1: ...(grocery2.geometry) AS grocery_count FROM airbnb, grocery2 W...
                                                             ^


In [28]:
#print the results of calculation in above cell
cur.execute("SELECT * FROM airbnb_grocery3 limit 5")
result = cur.fetchall()
print(results)

UndefinedTable: relation "airbnb_grocery3" does not exist
LINE 1: SELECT * FROM airbnb_grocery3 limit 5
                      ^


In [ ]:
cur.close()
con.commit()
con.close()

In [29]:
help(engine)

Help on Engine in module sqlalchemy.engine.base object:

class Engine(sqlalchemy.engine.interfaces.Connectable, sqlalchemy.log.Identified)
 |  Engine(pool, dialect, url, logging_name=None, echo=None, query_cache_size=500, execution_options=None, hide_parameters=False)
 |  
 |  Connects a :class:`~sqlalchemy.pool.Pool` and
 |  :class:`~sqlalchemy.engine.interfaces.Dialect` together to provide a
 |  source of database connectivity and behavior.
 |  
 |  This is the **SQLAlchemy 1.x version** of :class:`_engine.Engine`.  For
 |  the :term:`2.0 style` version, which includes  some API differences,
 |  see :class:`_future.Engine`.
 |  
 |  An :class:`_engine.Engine` object is instantiated publicly using the
 |  :func:`~sqlalchemy.create_engine` function.
 |  
 |  .. seealso::
 |  
 |      :doc:`/core/engines`
 |  
 |      :ref:`connections_toplevel`
 |  
 |  Method resolution order:
 |      Engine
 |      sqlalchemy.engine.interfaces.Connectable
 |      sqlalchemy.log.Identified
 |      bui